## Cheatsheet: scraping Wikipedia tables

I think we, as a society, have reached a consensus on Wikipedia being a pretty decent source of information. Assuming you, like me, wake up at 3 AM wondering who would be better on your Would I Lie to You? team (the correct answer should always be [Bob Mortimer](https://www.youtube.com/watch?v=MsuuiVzS6Js), regardless of whether you win or lose), being able to scrape Wikipedia tables can come in pretty handy.<sup>1</sup>

For now, let's just get the information. We can worry about running analysis on it later!

<sup>1</sup> <sub>Please note that Wikipedia is wonderful and provides a [free Python library that allows users to search and parse data from the site.](https://pypi.python.org/pypi/wikipedia/) There's no real need to overload their site with scrapers</sub>

In [1]:
#Let's start by loading a tonne of libraries that we need
#BeautifulSoup is a library that lets you scrape sites
#urllib is a package that lets you work with URLs
#pandas is the best thing that ever happened to me
import bs4
from bs4 import BeautifulSoup
import urllib
import pandas as pd

In [2]:
#Let's declare what the URL we're using is
url = "https://en.wikipedia.org/wiki/List_of_Would_I_Lie_to_You%3F_episodes"

In [3]:
#And then open it and read it using urllib
html = urllib.urlopen(url).read()

In [4]:
#This is what html looks like right now (but only the first 300 characters because otherwise your eyes start to cross)
html[0:300]
#Super useful.

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of Would I Lie to You? episodes - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script'

In [5]:
#We can use BeautifulSoup to clean this up a bit
soup = BeautifulSoup(html, "html.parser", from_encoding="utf-8")

In [6]:
for tag in soup.find_all(name="table", attrs={'class':'wikitable'}):
    print tag.text

    
#cool, so we can now see who the contestants are, when the episode was aired, and the scores (David's team on right and Lee's on the left)



Episode
First broadcast
David's team
Lee's team
Scores


01x01
16 June 2007
Duncan Bannatyne and Frankie Boyle
Natalie Cassidy and Dom Joly
11–11


01x02
23 June 2007
Patrick McGuinness and Fay Ripley
John Barrowman and Dominic Wood
6–10


01x03
30 June 2007
Eamonn Holmes and Dara Ó Briain
Jimmy Carr and Ulrika Jonsson
13–6


01x04
14 July 2007[fn 1]
Myleene Klass and Jason Manford
Leslie Ash and Neil Morrissey
7–11


01x05
21 July 2007
Russell Howard and Wendy Richard
Len Goodman and Vic Reeves
11–9


01x06
28 July 2007
Harry Enfield and Claudia Winkleman
Tara Palmer-Tomkinson and Dave Spikey
9–8




Episode
First broadcast
David's team
Lee's team
Scores


02x01
11 July 2008
Rob Brydon and Krishnan Guru-Murthy
Gabby Logan and Robert Webb
7–6


02x02
18 July 2008
Trisha Goddard and Rich Hall
Frankie Boyle and Ben Shephard
7–5


02x03
25 July 2008
David Baddiel and Maureen Lipman
Jimmy Carr and Richard Wilson
5–6


02x04
1 August 2008
Michael Aspel and Dara Ó Briain
Jason Manford and 

In [7]:
#let's figure out which of these rows and tables we want, because it looks like some include compilation videos
#to do that, let's first look for the <tr> tag (which is used to denote table rows on Wikipedia), and then look at
#the <td> tag, which is used to look at table data
for row in soup.find_all('tr'):
    cells = row.find_all('td')
    print len(cells)

1
0
5
5
5
5
5
5
0
5
5
5
5
5
5
5
5
4
0
5
5
5
5
5
5
5
5
4
0
5
5
5
5
5
5
5
5
4
0
5
0
5
5
5
5
5
5
5
5
4
0
5
5
5
5
5
5
5
5
4
0
5
5
5
5
5
5
5
5
4
5
0
5
5
5
5
5
5
5
5
5
4
0
5
5
5
5
5
5
5
5
5
4
0
5
5
5
5
5
5
5
5
4
5
0
5
2
1
2
1
2


In [8]:
#ok, so we probably only need the rows with 5 separate cells of information. 
#let's create several empty lists and then append the information from each cell to the list

number = []
date = []
david_team = []
lee_team = []
score = []

for entry in soup.find_all('tr'):
    cells = entry.find_all('td')
    if len(cells) == 5:
        number.append(cells[0].find(text=True))
        date.append(cells[1].find(text=True))
        david_team.append(cells[2].find_all(text=True))
        lee_team.append(cells[3].find_all(text=True))
        score.append(cells[4].find(text=True))

#using .find() for some and .find_all() for others is handy when you've got multiple values in a cell. 
#let's see how well that worked.

In [9]:
#I've converted everything to a dataframe and added column names
df = pd.DataFrame(data=[number, date, david_team, lee_team, score]).transpose()
df.columns = ["episode_no", "date_aired", "david_team", "lee_team", "score"]
df.head()

#hmm--i don't want to have lists within the columns for each team, however. 

,episode_no,date_aired,david_team,lee_team,score
0,01x01,16 June 2007,"[Duncan Bannatyne, and , Frankie Boyle]","[Natalie Cassidy, and , Dom Joly]",11–11
1,01x02,23 June 2007,"[Patrick McGuinness, and , Fay Ripley]","[John Barrowman, and , Dominic Wood]",6–10
2,01x03,30 June 2007,"[Eamonn Holmes, and , Dara Ó Briain]","[Jimmy Carr, and , Ulrika Jonsson]",13–6
3,01x04,14 July 2007,"[Myleene Klass, and , Jason Manford]","[Leslie Ash, and , Neil Morrissey]",7–11
4,01x05,21 July 2007,"[Russell Howard, and , Wendy Richard]","[Len Goodman, and , Vic Reeves]",11–9


In [10]:
#let's try a different function
number = []
date = []
david_team = []
lee_team = []
score = []

for entry in soup.find_all('tr'):
    cells = entry.find_all('td')
    if len(cells) == 5:
        number.append(cells[0].get_text())
        date.append(cells[1].get_text())
        david_team.append(cells[2].get_text())
        lee_team.append(cells[3].get_text())
        score.append(cells[4].get_text())
        

In [11]:
df = pd.DataFrame(data=[number, date, david_team, lee_team, score]).transpose()

In [12]:
df.columns = ["episode_no", "date_aired", "david_team", "lee_team", "score"]

In [13]:
df.head()
#that's much better!

,episode_no,date_aired,david_team,lee_team,score
0,01x01,16 June 2007,Duncan Bannatyne and Frankie Boyle,Natalie Cassidy and Dom Joly,11–11
1,01x02,23 June 2007,Patrick McGuinness and Fay Ripley,John Barrowman and Dominic Wood,6–10
2,01x03,30 June 2007,Eamonn Holmes and Dara Ó Briain,Jimmy Carr and Ulrika Jonsson,13–6
3,01x04,14 July 2007[fn 1],Myleene Klass and Jason Manford,Leslie Ash and Neil Morrissey,7–11
4,01x05,21 July 2007,Russell Howard and Wendy Richard,Len Goodman and Vic Reeves,11–9


That's a clean dataframe that I can now work with to figure out if having Bob Mortimer on your team is in fact the winning formula for WILTY.

You're welcome, BBC.